<a href="https://colab.research.google.com/github/phiriv/Intro-to-ML-Safety/blob/main/concept_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/neelnanda-io/Easy-Transformer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/neelnanda-io/Easy-Transformer.git to /tmp/pip-req-build-hdr3urdt
  Running command git clone -q https://github.com/neelnanda-io/Easy-Transformer.git /tmp/pip-req-build-hdr3urdt


In [ ]:
# Giles code
import torch
from easy_transformer import EasyTransformer
import json
import numpy as np

short = 100

n_layers = 12
mlp_layer_size = 3072
vocab_size = 50257
num_to_display = 2
max_display_tokens = 20

attention_cache = [None] * n_layers

def attention_hook(neuron_acts, hook, layer):
    attention_cache[layer] = neuron_acts[0,:,:,:].to('cpu')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")
torch.set_grad_enabled(False)

model = EasyTransformer.from_pretrained('gpt2').to(device)

for layer in range(n_layers):
    model.blocks[layer].attn.hook_attn.add_hook(lambda a,hook,layer=layer: attention_hook(a,hook,layer))
print("Added hooks")

def get_attentions_for_prompt(prompt):
  global attention_cache
  prompt_tokens = model.to_tokens(prompt, prepend_bos=True)
  prompt_tokens_str = [model.tokenizer.decode(t) for t in prompt_tokens[0]]
  attention_cache = [None] * n_layers
  logits = model(prompt_tokens)[0,:,:].to('cpu')
  probs = torch.nn.functional.softmax(logits, dim=1)
  if attention_cache[0] == None:
    raise Exception("None found in attention_cache")
  activs = list(attention_cache)
  return activs, prompt_tokens_str


Using cuda device
Loading model: gpt2


Using pad_token, but it is not set yet.


Moving model to device:  cuda
Finished loading pretrained model gpt2 into EasyTransformer!
Moving model to device:  cuda
Added hooks


In [ ]:
# Install an older version of node, so that Svelte works (a web dev framework)
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
!pip install git+https://github.com/neelnanda-io/PySvelte.git
import sys
sys.path.append('/content/PySvelte')


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://deb.nodesource.com/node_16.x bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRel

In [ ]:
import pysvelte

prompt1 = "The bananas grow on"
prompt2 = "The pigs grow on"

a1, t1 = get_attentions_for_prompt(prompt1)
a2, t2 = get_attentions_for_prompt(prompt2)

layer_num = 3
pysvelte.AttentionMulti(tokens=t1, attention=a1[layer_num].permute(1, 2, 0))

In [ ]:
pysvelte.AttentionMulti(tokens=t2, attention=a2[layer_num].permute(1, 2, 0))

In [ ]:
pysvelte.AttentionMulti(tokens=t2, attention=torch.abs(a1[layer_num] - a2[layer_num]).permute(1, 2, 0))

In [ ]:
# Find which layers/heads have particularly large differences either way
for layer in range(12):
  for head in range(12):
    diffs = a1[layer][head,:,:] - a2[layer][head,:,:]
    s = []
    for x in range(1,len(t1)):
      for y in range(1,len(t1)):
        if abs(diffs[x,y].item()) > 0.1:
          s.append((diffs[x,y].item(), t1[y], t1[x]))
    s.sort()
    if len(s) > 0:
      print(f'layer={layer}', f'head={head}', s)

layer=0 head=0 [(-0.19842997193336487, ' bananas', ' grow')]
layer=0 head=6 [(-0.1251545250415802, ' bananas', ' bananas'), (-0.10393300652503967, ' bananas', ' grow'), (0.1259615123271942, 'The', ' bananas')]
layer=0 head=7 [(-0.16453631222248077, ' bananas', ' grow')]
layer=0 head=8 [(0.10651837289333344, ' bananas', ' grow')]
layer=1 head=6 [(-0.137222558259964, ' bananas', ' bananas'), (0.12932801246643066, ' bananas', ' grow'), (0.15488284826278687, 'The', ' bananas')]
layer=2 head=0 [(0.10848182439804077, ' bananas', ' grow')]
layer=2 head=2 [(-0.1732567548751831, 'The', ' bananas')]
layer=2 head=3 [(0.1431485414505005, ' bananas', ' on')]
layer=2 head=5 [(0.1650000512599945, ' bananas', ' grow')]
layer=2 head=8 [(0.21858426928520203, ' bananas', ' grow')]
layer=3 head=2 [(-0.20532464981079102, 'The', ' bananas'), (-0.14521878957748413, ' grow', ' on')]
layer=3 head=5 [(-0.11927641183137894, ' bananas', ' grow')]
layer=3 head=7 [(0.12615281343460083, ' bananas', ' grow')]
layer=3